In [1]:
import xgboost as xgb
import pandas as pd
from sklearn.metrics import f1_score

ModuleNotFoundError: No module named 'xgboost'

In [ ]:
train_features = pd.read_csv('dataset1_train_features.csv')
test_features = pd.read_csv('dataset1_test_features.csv')

train_types = []

for row in train_features['Type']:
    if row == 'Class':
        train_types.append(1)
    else:
        train_types.append(0)
        
train_features['Type_encode'] = train_types

test_types = []

for row in test_features['Type']:
    if row == 'Class':
        test_types.append(1)
    else:
        test_types.append(0)
        
test_features['Type_encode'] = test_types

X_train = train_features.loc[:, 'Ngram1_Entity':'Type_encode']
y_train = train_features['Match']

X_test = test_features.loc[:, 'Ngram1_Entity':'Type_encode']
y_test = test_features['Match']

df_train = train_features.loc[:, 'Ngram1_Entity':'Type_encode']
df_train['Match'] = train_features['Match']

df_test = test_features.loc[:, 'Ngram1_Entity':'Type_encode']
df_test['Match'] = test_features['Match']

X_train = X_train.fillna(value=0)
X_test = X_test.fillna(value=0)

In [4]:
min_child_weight_s = [1, 5, 10]
gamma_s = [0.5, 1, 1.5, 2, 5]
subsample_s = [0.6, 0.8, 1.0]
colsample_bytree_s = [0.6, 0.8, 1.0]
max_depth_s = [3, 4, 5]

In [5]:
def f1_eval(y_pred, dtrain):
    y_true = dtrain.get_label()
    err = 1-f1_score(y_true, np.round(y_pred))
    return 'f1_err', err

In [ ]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

data = []

for min_child_weight in min_child_weight_s:
    for gamma in gamma_s:
        for subsample in subsample_s:
            for colsample_bytree in colsample_bytree_s:
                for max_depth in max_depth_s:

                    param = {'silent': 0,
                             'objective': 'binary:logistic',
                             'min_child_weight': min_child_weight,
                             'gamma': gamma,
                             'subsample': subsample,
                             'colsample_bytree': colsample_bytree,
                             'max_depth': max_depth
                            }
                    param['nthread'] = 4
                    param['eval_metric'] = f1_eval

                    evallist = [(dtest, 'eval'), (dtrain, 'train')]

                    plst = param.items()

                    num_round = 10
                    bst = xgb.train(plst, dtrain, num_round, evallist, verbose_eval=False)
                    
                    y_pred = bst.predict(dtest)
                    f1 = f1_score(y_test, y_pred)
                    
                    print(min_child_weight, gamma, subsample, colsample_bytree, max_depth)
                    data.append((min_child_weight, gamma, subsample, colsample_bytree, max_depth, f1))

                    
                    
                    
dataset = pd.DataFrame(data, columns=['min_child_weight', 'gamma', 'subsample',
                                      'colsample_bytree', 'max_depth', 'f1'])

dataset.to_csv('dataset1_xgboost.csv', index=False)